# Ukázka dashboard

Matěj Kolář 2022

UHK - FIM

## Sdílený kód

Zdroj dat: https://www.kaggle.com/datasets/lobosi/c02-emission-by-countrys-grouth-and-population?resource=download (Pod licencí DbCL v1.0)

In [2]:
import pandas as pd

ENERGY_TYPE = ["all_energy_types", "coal", "natural_gas", "petroleum_n_other_liquids", "nuclear","renewables_n_other"]
ENERGY_TYPE_HUMAN = ["Celkově", "Uhlí","Plyn","Petrolej, tekutá paliva","Jádro","Obnovitelné a ostatní"]

#načte data
df = pd.read_csv("../data/energy.csv",index_col=0)

# rozdíl produkce a spotřeby
def diff(row):
    return row["Energy_production"] - row["Energy_consumption"]
df["Diff"] = df.apply(diff,axis=1)

def co_per_btu(row):
    if row["Energy_type"] == "all_energy_types" and row["Energy_production"] > 0:
        return row["CO2_emission"]/row["Energy_production"]
df["CO2_per_BTU"] = df.apply(co_per_btu,axis=1)

def co_per_cap(row):
    if row["Energy_type"] == "all_energy_types" and row["Population"] > 0:
        return row["CO2_emission"]/row["Population"]
df["CO2_per_capita"] = df.apply(co_per_cap,axis=1)

class Proc_help():
    def __init__(self):
        self.year = 0
        self.country = 0
    def proc(self,row):
        year = int(row["Year"])
        country = str(row["Country"])
        if self.year != year or self.country != country:
            self.power = list(df.query("Year == @year & Country == @country")["Energy_production"])[0]
            self.year = year
            self.country = country
        if self.power > 0:
            return (row["Energy_production"] / self.power)*100
        else:
            return 0
        
proc = Proc_help()
# % prudukce
df["Proc"] = df.apply(proc.proc,axis=1)


def proces_data(country, year):
    # Data process 
    data = df[df["Country"] == country]
    data = data.query("Year >= @year[0] & Year <= @year[1]")
    data_typed = {}
    for type in ENERGY_TYPE:
        data_typed[type] = data[data["Energy_type"] == type]
    return data_typed, data

## Matplotlib

In [3]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec
import panel as pn  

def create_figure(country, year, co2):
    with plt.style.context("fast","ggplot"):
        fig = plt.figure(figsize=(20,8))
        specs = gridspec.GridSpec(ncols=2, nrows=4, figure=fig, hspace = 0.75, wspace=0.15)

        ax1 = fig.add_subplot(specs[0:2, 0])
        ax2 = fig.add_subplot(specs[2:4, 0])
        ax3 = fig.add_subplot(specs[0, 1])
        ax4 = fig.add_subplot(specs[1, 1])
        
        # Data process 
        data_typed, _ = proces_data(country, year)
        
        # Produkce
        ax1.plot(data_typed["all_energy_types"]["Year"], data_typed["all_energy_types"]["Energy_production"])
        ax1.set_xlabel("Rok")
        ax1.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        ax1.set_ylabel("quad BTU")
        ax1.set_title("Produkce v historii")

        # Zdroje
        dd = []
        labels = []
        for type in ENERGY_TYPE[1:]:
            dd.append(data_typed[type]["Proc"])
            labels.append(ENERGY_TYPE_HUMAN[ENERGY_TYPE.index(type)])

        ax2.stackplot(data_typed["all_energy_types"]["Year"],*dd, labels=labels)
        ax2.set_title("Procentuální rozdělení produkce")
        ax2.set_xlabel("Rok")
        ax2.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        ax2.set_ylabel("%")
        ax2.legend(loc="upper left")
        
        # Přebytek
        ax3.plot(data_typed["all_energy_types"]["Year"], data_typed["all_energy_types"]["Diff"])
        ax3.set_title("Přebytek")
        ax3.set_xlabel("Rok")
        ax3.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
        ax3.set_ylabel("quad BTU")
        
        # CO2
        if co2 == "celkem":
            ax4.plot(data_typed["all_energy_types"]["Year"], data_typed["all_energy_types"]["CO2_emission"])
            ax4.set_title("Celková produkce CO2")
        elif co2 == "za quad BTU":
            ax4.plot(data_typed["all_energy_types"]["Year"], data_typed["all_energy_types"]["CO2_per_BTU"])
            ax4.set_title("Vyprodukované CO2 za quadBTU")
        elif co2 == "na obyvatele":
            ax4.plot(data_typed["all_energy_types"]["Year"], data_typed["all_energy_types"]["CO2_per_capita"])
            ax4.set_title("Vyprodukované CO2 na obyvatele")
        ax4.set_ylabel("MMtones CO2")
        ax4.set_xlabel("Rok")
        ax4.xaxis.set_major_locator(ticker.MaxNLocator(integer=True))
            
        plt.close(fig)
    return fig

pn.extension()
h1 = pn.pane.Markdown("""# Energie v historii""",align="center")
locations = list(df["Country"].unique())
year = pn.widgets.IntRangeSlider(name="Rok", width=250, start=min(df["Year"]), end=max(df["Year"]), value=(min(df["Year"]), max(df["Year"])))
country = pn.widgets.Select(name="Oblast", options=locations)
co2_label = pn.pane.Markdown("""Produkce CO2""",margin=(-8,0,-18,10))
co2 = pn.widgets.RadioButtonGroup(name="Produkce CO2", options=["celkem", "za quad BTU", "na obyvatele"])
kw = dict(country = country,
          year = year,
          co2 = co2)

co2_combined = pn.Column(co2_label,co2)
dash = pn.interact(create_figure, **kw)
inputs = pn.Row(country, year, co2_combined,align="center")
pn.Column(h1, inputs, dash[1],align="center")

BokehModel(combine_events=True, render_bundle={'docs_json': {'0cbfb363-6267-414a-9a56-f16e245fae2f': {'defs': …

## Plotly

In [4]:
from dash import Dash, dcc, html, Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

locations = list(df["Country"].unique())

app = JupyterDash(__name__)

input_style = {"padding": 10, "flex": 1}
graph_style = {"padding": 0, }
country = html.Div(children=[html.Label("Oblast"),dcc.Dropdown(locations, "World", id="country")], style=input_style)
year = html.Div(children=[html.Label("Rok"),dcc.RangeSlider(min=min(df["Year"]), max=max(df["Year"]), step=1, 
                                                            tooltip={"placement": "bottom", "always_visible": True}, value=(min(df["Year"]), max(df["Year"])), 
                                                            marks=None, id = "year")],style=input_style)
co2 = html.Div(children=[html.Label("Produkce CO2"),dcc.RadioItems(["celkem", "za quad BTU", "na obyvatele"], "celkem", id = "co2")],style=input_style)

inputs = html.Div([country, year,co2],style={"display": "flex", "flex-direction": "row"})

app.layout = html.Div([
    html.Center(html.H1("Energie v historii")),
    inputs,
    dcc.Graph(id="plot", style=graph_style),],style={"backgroundColor": "rgb(255, 255, 255)"}
)

@app.callback(
    Output("plot", "figure"),
    Input("country", "value"),Input("year", "value"),Input("co2", "value")
)
def create_figure(country, year, co2):
    data_typed, _ = proces_data(country, year)
    specs=[[{"rowspan": 2}, {}],
           [None, {}],
           [{"rowspan": 2}, {}],
           [None, {}]
           ]
    plot = make_subplots(rows=4, cols=2, specs=specs,subplot_titles=("Produkce v historii", "Přebytek", "CO2", "Procentuální rozdělení produkce"),vertical_spacing=0.15)
    
    # Produkce
    labels={"x":"Rok","y":"qBTU"}
    fig1 = px.line(x=data_typed["all_energy_types"]["Year"], y=data_typed["all_energy_types"]["Energy_production"], title="Produkce v historii",labels=labels)
    
    # Zdroje
    fig2 = go.Figure()
    for type in ENERGY_TYPE[1:]:
        if max(data_typed[type]["Proc"]) > 0.1:
            fig2.add_trace(go.Scatter(x=data_typed["all_energy_types"]["Year"], y=data_typed[type]["Proc"],
                                stackgroup="one"))
            fig2.data[-1].name = ENERGY_TYPE_HUMAN[ENERGY_TYPE.index(type)]
    fig2.update_layout(showlegend=True,yaxis=dict(type="linear",ticksuffix="%"),title="Procentuální rozdělení produkce")
    
    # Přebytek
    labels={"x":"Rok","y":"qBTU"}
    fig3 = px.line(x=data_typed["all_energy_types"]["Year"], y=data_typed["all_energy_types"]["Diff"], title = "Přebytek",labels=labels)

    # CO2
    labels={"x":"Rok","y":"MMtones CO2"}
    if co2 == "celkem":
        fig4 = px.line(x=data_typed["all_energy_types"]["Year"], y=data_typed["all_energy_types"]["CO2_emission"], title="Celková produkce CO2",labels=labels)
    elif co2 == "za quad BTU":
        fig4 = px.line(x=data_typed["all_energy_types"]["Year"], y=data_typed["all_energy_types"]["CO2_per_BTU"], title="Vyprodukované CO2 za quadBTU",labels=labels)
    elif co2 == "na obyvatele":
        fig4 = px.line(x=data_typed["all_energy_types"]["Year"], y=data_typed["all_energy_types"]["CO2_per_capita"], title="Vyprodukované CO2 na obyvatele",labels=labels)
    
    
    fig1_traces = []
    fig2_traces = []
    fig3_traces = []
    fig4_traces = []
    for trace in range(len(fig1["data"])):
        fig1_traces.append(fig1["data"][trace])
    for trace in range(len(fig2["data"])):
        fig2_traces.append(fig2["data"][trace])
    for trace in range(len(fig3["data"])):
        fig3_traces.append(fig3["data"][trace])
    for trace in range(len(fig4["data"])):
        fig4_traces.append(fig4["data"][trace])
        
    for traces in fig1_traces:
        plot.append_trace(traces, row=1, col=1)
    for traces in fig2_traces:
        plot.append_trace(traces, row=3, col=1)
    for traces in fig3_traces:
        plot.append_trace(traces, row=1, col=2)
    for traces in fig4_traces:
        plot.append_trace(traces, row=2, col=2)
        
        
    plot.update_xaxes(title_text="Rok", row = 1, col = 1, title_standoff = 1)    
    plot.update_xaxes(title_text="Rok", row = 3, col = 1, title_standoff = 1)  
    plot.update_xaxes(title_text="Rok", row = 1, col = 2, title_standoff = 1)  
    plot.update_xaxes(title_text="Rok", row = 2, col = 2, title_standoff = 1)  
    
    plot.update_yaxes(title_text="qBTU", row = 1, col = 1)    
    plot.update_yaxes(title_text="", row = 3, col = 1, ticksuffix="%")  
    plot.update_yaxes(title_text="qBTU", row = 1, col = 2)  
    plot.update_yaxes(title_text="MMtones", row = 2, col = 2)  
    
    plot.update_layout(margin={"r":0,"t":20,"l":0,"b":0})
    return plot

app.run_server(mode="inline")

## Vega

In [6]:
import altair as alt
import panel as pn  

def create_figure(country, year, co2):
    # Data process 
    data_typed, data = proces_data(country, year)
    
    year_format =alt.Y("Year:Q",axis=alt.Axis(title="Rok", format=".0f"))
    
    # Produkce
    fig1 = alt.Chart(data_typed["all_energy_types"]).mark_line().encode(x=year_format,y=alt.Y("Energy_production",axis=alt.Axis(title="qBTU")))

    # Zdroje  
    data = data[data["Energy_type"] != "all_energy_types"]
    data = data[data["Proc"] > 0.1]
    
    legend=alt.Legend(title=None)
    fig2 = alt.Chart(data).mark_area().encode(x=year_format,y=alt.Y("sum(Proc):Q",axis=alt.Axis(title="%")),color=alt.Color("Energy_type",legend=legend))
    
    #Přebytek
    fig3 = alt.Chart(data_typed["all_energy_types"]).mark_line().encode(x=year_format,y=alt.Y("Diff",axis=alt.Axis(title="qBTU")))
    
    # CO2
    co2_y_axis = alt.Axis(title="MMtones CO2")
    if co2 == "celkem":
        fig4 = alt.Chart(data_typed["all_energy_types"]).mark_line().encode(x=year_format,y=alt.Y("CO2_emission",axis=co2_y_axis))
    elif co2 == "za quad BTU":
        fig4 = alt.Chart(data_typed["all_energy_types"]).mark_line().encode(x=year_format,y=alt.Y("CO2_per_BTU",axis=co2_y_axis))
    elif co2 == "na obyvatele":
        fig4 = alt.Chart(data_typed["all_energy_types"]).mark_line().encode(x=year_format,y=alt.Y("CO2_per_capita",axis=co2_y_axis))
        
    col_0 = alt.vconcat(fig1,fig2)
    col_1 = alt.vconcat(fig3,fig4)
    return alt.hconcat(col_0,col_1)

pn.extension()
h1 = pn.pane.Markdown("""# Energie v historii""",align="center")
locations = list(df["Country"].unique())
year = pn.widgets.IntRangeSlider(name="Rok", width=250, start=min(df["Year"]), end=max(df["Year"]), value=(min(df["Year"]), max(df["Year"])))
country = pn.widgets.Select(name="Oblast", options=locations)
co2_label = pn.pane.Markdown("""Produkce CO2""",margin=(-8,0,-18,10))
co2 = pn.widgets.RadioButtonGroup(name="Produkce CO2", options=["celkem", "za quad BTU", "na obyvatele"])
kw = dict(country = country,
          year = year,
          co2 = co2)

co2_combined = pn.Column(co2_label,co2)
dash = pn.interact(create_figure, **kw)
inputs = pn.Row(country, year, co2_combined,align="center")
pn.Column(h1, inputs, dash[1])

BokehModel(combine_events=True, render_bundle={'docs_json': {'eb5582e4-28d6-4da9-a750-1dc1f3a5e2fd': {'defs': …